### Compute respiration signal for inspiration without odor

In [5]:
from connection import *
from collections import OrderedDict
from pandas import DataFrame, Series, concat, ExcelWriter,merge, MultiIndex
from score_common import create_df_subject
from score_encoding import create_df_subject_encode
from itertools import product
import numpy as np
import matplotlib.pyplot as plt

exp = 'E'
path2save = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/respiration_amplitude/'

for subject in session.query(Subject).order_by('Subject.`group`'):
    print subject.name
    times_all, resp_all, inspi_no_odor = np.array([]),np.array([]), []
    for run in subject.runs.filter_by(exp =exp).order_by('`index`'):
        #Compute timings of inspiration without odors
        resp = run.respirationsignals[0]
        inspi_times = resp.cycle_times[:, 0]
        inspi_times = inspi_times.astype(float64)
        print(inspi_times.shape,inspi_times.dtype)
        odor_times = np.array([])
        for trial in run.trials:
            odor_times = np.hstack((odor_times,trial.first_inspiration_time)) if np.size(odor_times) else trial.first_inspiration_time
        print(odor_times.shape)
        inspi_no_odor = []
        for time_insp in inspi_times:
            if time_insp.item() not in odor_times and time_insp.item() > 0: #pour ne pas avoir de valeurs négatives (vu qu'onfait -2s)
                inspi_no_odor = np.append(inspi_no_odor,time_insp)
        inspi_no_odor = np.array(inspi_no_odor[1:-2])
        print('no odor times',inspi_no_odor.shape)
        
        #Compute mean inspiration signal for all inspi without odors
        sig = resp.signal
        t_start = resp.t_start.rescale('s').magnitude
        sr = resp.sampling_rate.rescale('Hz').magnitude
        times = np.arange(sig.shape[0])/sr + t_start
        for time in inspi_no_odor:
            t1 = time
            t2 = time + 2.
            sig_chunk = sig[(times>=t1) & (times<=t2)][np.newaxis]*-1
            times_chunk = times[(times>=t1) & (times<=t2)]-t1
            times_chunk = times_chunk[np.newaxis]
            if sig_chunk.shape[1] == 199:
                sig_chunk = np.append(0,sig_chunk)[np.newaxis]
                times_chunk = np.append(0,times_chunk)[np.newaxis]
            if sig_chunk.shape[1] == 201:
                sig_chunk = np.delete(sig_chunk,[0],axis=1)
                times_chunk = np.delete(times_chunk,[0],axis=1)
            #print(np.min(times_chunk), np.max(times_chunk))
            #plt.plot(np.squeeze(times_chunk),np.squeeze(sig_chunk))
            #print('single trial',sig_chunk.shape,times_chunk.shape)
            resp_all = np.vstack((resp_all,sig_chunk)) if np.size(resp_all) else sig_chunk
            times_all = np.vstack((times_all,times_chunk)) if np.size(times_all) else times_chunk
        print('all trials',resp_all.shape,times_all.shape)
        np.save(path2save+subject.name+'_time_'+exp+'_no_odor_resp.npy',np.squeeze(times_chunk))
        np.save(path2save+subject.name+'_sig_'+exp+'_no_odor_resp.npy',np.squeeze(resp_all))

CHAF
((114,), dtype('float64'))
(39,)
('no odor times', (73,))
('all trials', (73, 200), (73, 200))
((92,), dtype('float64'))
(34,)
('no odor times', (55,))
('all trials', (128, 200), (128, 200))
VACJ
((170,), dtype('float64'))
(15,)
('no odor times', (153,))
('all trials', (153, 200), (153, 200))
((209,), dtype('float64'))
(17,)
('no odor times', (173,))
('all trials', (326, 200), (326, 200))
SEMC
((136,), dtype('float64'))
(13,)
('no odor times', (123,))
('all trials', (123, 200), (123, 200))
((140,), dtype('float64'))
(28,)
('no odor times', (110,))
('all trials', (233, 200), (233, 200))
LEFC
((155,), dtype('float64'))
(16,)
('no odor times', (136,))
('all trials', (136, 200), (136, 200))
((163,), dtype('float64'))
(45,)
('no odor times', (116,))
('all trials', (252, 200), (252, 200))
FERJ
((161,), dtype('float64'))
(14,)
('no odor times', (144,))
('all trials', (144, 200), (144, 200))
((181,), dtype('float64'))
(42,)
('no odor times', (137,))
('all trials', (281, 200), (281, 200))


In [ ]:
from connection import *
from collections import OrderedDict
from pandas import DataFrame, Series, concat, ExcelWriter,merge, MultiIndex
from score_common import create_df_subject
from score_encoding import create_df_subject_encode

import numpy as np
import matplotlib.pyplot as plt

path2save = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/respiration_amplitude/'
exp = 'R'
for subject in session.query(Subject).order_by('Subject.`group`'):
    times_all, resp_all = np.array([]),np.array([])
    print subject.name
    for run in subject.runs.filter_by(exp =exp).order_by('`index`'):
        resp = run.respirationsignals[0]
        sig = resp.signal
        t_start = resp.t_start.rescale('s').magnitude
        sr = resp.sampling_rate.rescale('Hz').magnitude
        times = np.arange(sig.shape[0])/sr + t_start
        for trial in run.trials:
            t1 = trial.time + 1.5
            t2 = trial.time + 3.5
            sig_chunk = sig[(times>=t1) & (times<=t2)][np.newaxis]*-1
            times_chunk = times[(times>=t1) & (times<=t2)]-t1
            times_chunk = times_chunk[np.newaxis]
            if sig_chunk.shape[1] == 199:
                sig_chunk = np.append(0,sig_chunk)[np.newaxis]
                times_chunk = np.append(0,times_chunk)[np.newaxis]
            if sig_chunk.shape[1] == 201:
                sig_chunk = np.delete(sig_chunk,[0],axis=1)
                times_chunk = np.delete(times_chunk,[0],axis=1)
            #plt.plot(np.squeeze(times_chunk),np.squeeze(sig_chunk))
            #print('single trial',sig_chunk.shape,times_chunk.shape)
            resp_all = np.vstack((resp_all,sig_chunk)) if np.size(resp_all) else sig_chunk
            times_all = np.vstack((times_all,times_chunk)) if np.size(times_all) else times_chunk
    #plt.plot(np.mean(times_all,axis=0),np.mean(resp_all,axis=0))
    #plt.show()
    print('all trials',resp_all.shape,times_all.shape)
    np.save(path2save+subject.name+'_time_'+exp+'_resp.npy',np.squeeze(times_chunk))
    np.save(path2save+subject.name+'_sig_'+exp+'_resp.npy',np.squeeze(resp_all))

### Timing for respiration

In [ ]:
from connection import *
from collections import OrderedDict
from pandas import DataFrame, Series, concat, ExcelWriter,merge, MultiIndex
from score_common import create_df_subject
from score_encoding import create_df_subject_encode
from itertools import product
import numpy as np
import matplotlib.pyplot as plt

exp = 'E'

for subject in session.query(Subject).order_by('Subject.`group`'):
    print subject.name
    for run in subject.runs.filter_by(exp =exp).order_by('`index`'):
        resp = run.respirationsignals[0]
        inspi_times = resp.cycle_times[:, 0]
        inspi_times = inspi_times.astype(float64)
        print(inspi_times.shape,inspi_times.dtype)
        odor_times = np.array([])
        for trial in run.trials:
            odor_times = np.hstack((odor_times,trial.first_inspiration_time)) if np.size(odor_times) else trial.first_inspiration_time
        print(odor_times.shape)
        inspi_no_odor = []
        for time_insp in inspi_times:
            if time_insp.item() not in odor_times and time_insp.item() > 2: #pour ne pas avoir de valeurs négatives (vu qu'onfait -2s)
                inspi_no_odor = np.append(inspi_no_odor,time_insp)
        inspi_no_odor = np.array(inspi_no_odor)
        print('no odor times',inspi_no_odor.shape)